# 设置数据文件存放根目录（不直接放在该项目中，该项目有github仓库，避免大内存数据影响代码推送）

In [ ]:
boot_file_adr = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST"

# 建立映射表

In [ ]:
import pandas as pd
import os

# ========================
# 路径配置
# ========================
# boot_file_adr = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST"
raw_csv_dir = os.path.join(boot_file_adr, "raw_csv")
save_dir = os.path.join(boot_file_adr, "mapping_table")
os.makedirs(save_dir, exist_ok=True)
save_path = os.path.join(save_dir, "KAIST_vibration_mapping.csv")

# ========================
# 获取文件列表
# ========================
csv_files = sorted([f for f in os.listdir(raw_csv_dir) if f.endswith(".csv")])

# ========================
# 标签映射（Normal=1，不同转速/故障独立编号）
# ========================
mapping_rules = {
    "Normal": 1,
    "BPFI_03": 2,
    "BPFI_10": 3,
    "BPFI_30": 4,
    "BPFO_03": 5,
    "BPFO_10": 6,
    "BPFO_30": 7,
    "Misalign_01": 8,
    "Misalign_03": 9,
    "Misalign_05": 10,
    "Unbalance_0583mg": 11,
    "Unbalance_1169mg": 12,
    "Unbalance_1751mg": 13,
    "Unbalance_2239mg": 14,
    "Unbalance_3318mg": 15,
}

# ========================
# 生成映射表
# ========================
records = []
for f in csv_files:
    matched_label = None
    for key, label in mapping_rules.items():
        if key in f:
            matched_label = label
            break
    if matched_label is None:
        matched_label = -1
    records.append({
        "filename": f,
        "label": matched_label
    })

df = pd.DataFrame(records)
df.to_csv(save_path, index=False)

print(f"✅ 已生成映射表：{save_path}")
display(df)


# 数据混合

## 全类别混合，用于基本模型训练评估

### mixed data->csv

In [1]:
import os
import pandas as pd

# ===================== 1️⃣ 参数设置 =====================
raw_dir = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/raw_csv"
mapping_path = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mapping_table/KAIST_vibration_mapping.csv"
save_path = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/all_class_morenode/mixed_allclass.csv"
used_index_path = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/all_class_morenode//used_indices.csv"

# ========== 你需要的每类抽取数量 ==========
sample_dict = {
    1: 5000,
    2: 5000,
    3: 5000,
    4: 5000,
    5: 5000,
    6: 5000,
    7: 5000,
    8: 5000,
    9: 5000,
    10: 5000,
    11: 5000,
    12: 5000,
    13: 5000,
    14: 5000,
}

# ===================== 2️⃣ 读取映射表 =====================
mapping = pd.read_csv(mapping_path)
print(f"📘 映射表已读取，共 {len(mapping)} 条记录")

# ===================== 3️⃣ 加载已使用索引 =====================
if os.path.exists(used_index_path):
    used_indices = pd.read_csv(used_index_path)
else:
    used_indices = pd.DataFrame(columns=["source_file", "index", "label"])

# ===================== 4️⃣ 初始化输出文件 =====================
if os.path.exists(save_path):
    os.remove(save_path)
    print("⚙️ 已清空旧的 mixed_allclass.csv")
first_write = True

# ===================== 5️⃣ 实际抽样开始 =====================
# 遍历每一类
for label, target_n in sample_dict.items():

    print(f"\n==============================")
    print(f"🎯 开始处理类别 {label}，目标抽取 {target_n} 条")
    print(f"==============================")

    # 统计当前类别已抽取多少
    already_used = used_indices.loc[used_indices["label"] == label].shape[0]
    print(f"📊 当前类别 {label} 已有 {already_used} 条")

    if already_used >= target_n:
        print(f"✅ 类别 {label} 样本已足够（{already_used} >= {target_n}），跳过本类。")
        continue

    # 还需要抽的数
    remain = target_n - already_used
    print(f"🔍 还需 {remain} 条样本")

    # 遍历所有文件，属于该类的都尝试抽取
    for fname in os.listdir(raw_dir):

        if not fname.endswith(".csv"):
            continue
        if "2Nm" in fname:   # 排除 2Nm 文件
            continue

        # 找当前文件的标签
        entry = mapping.loc[mapping["filename"] == fname]
        if entry.empty:
            continue

        file_label = int(entry["label"].values[0])
        if file_label != label:
            continue

        file_path = os.path.join(raw_dir, fname)
        df = pd.read_csv(file_path)
        df["label"] = label
        df["source_file"] = fname

        # 当前文件未使用的样本
        used_idx_this_file = used_indices.loc[used_indices["source_file"] == fname, "index"].tolist()
        df_avail = df.loc[~df.index.isin(used_idx_this_file)]

        if df_avail.empty:
            print(f"⚠️ {fname} 可用数据为 0，跳过。")
            continue

        # 本文件可抽取数量
        take_n = min(remain, len(df_avail))

        sampled_df = df_avail.sample(n=take_n, random_state=42)

        # 写入输出文件
        sampled_df.to_csv(save_path, mode="a", header=first_write, index=False)
        first_write = False

        # 更新 used_indices
        new_used = pd.DataFrame({
            "source_file": fname,
            "index": sampled_df.index,
            "label": label
        })
        used_indices = pd.concat([used_indices, new_used], ignore_index=True)

        print(f"📦 从 {fname} 抽取 {take_n} 条。")

        remain -= take_n
        if remain <= 0:
            print(f"🎉 类别 {label} 已抽满 {target_n} 条。")
            break

# ===================== 6️⃣ 保存使用记录 =====================
used_indices.to_csv(used_index_path, index=False)
print("\n==============================")
print(f"🏁 所有类别抽样完成！")
print(f"📁 已保存：{save_path}")
print(f"📁 使用记录：{used_index_path}")
print("==============================")


📘 映射表已读取，共 15 条记录

🎯 开始处理类别 1，目标抽取 5000 条
📊 当前类别 1 已有 0 条
🔍 还需 5000 条样本
📦 从 0Nm_Normal.csv 抽取 5000 条。
🎉 类别 1 已抽满 5000 条。

🎯 开始处理类别 2，目标抽取 5000 条
📊 当前类别 2 已有 0 条
🔍 还需 5000 条样本
📦 从 0Nm_BPFI_03.csv 抽取 5000 条。
🎉 类别 2 已抽满 5000 条。

🎯 开始处理类别 3，目标抽取 5000 条
📊 当前类别 3 已有 0 条
🔍 还需 5000 条样本
📦 从 0Nm_BPFI_10.csv 抽取 5000 条。
🎉 类别 3 已抽满 5000 条。

🎯 开始处理类别 4，目标抽取 5000 条
📊 当前类别 4 已有 0 条
🔍 还需 5000 条样本
📦 从 0Nm_BPFI_30.csv 抽取 5000 条。
🎉 类别 4 已抽满 5000 条。

🎯 开始处理类别 5，目标抽取 5000 条
📊 当前类别 5 已有 0 条
🔍 还需 5000 条样本
📦 从 0Nm_BPFO_03.csv 抽取 5000 条。
🎉 类别 5 已抽满 5000 条。

🎯 开始处理类别 6，目标抽取 5000 条
📊 当前类别 6 已有 0 条
🔍 还需 5000 条样本
📦 从 0Nm_BPFO_10.csv 抽取 5000 条。
🎉 类别 6 已抽满 5000 条。

🎯 开始处理类别 7，目标抽取 5000 条
📊 当前类别 7 已有 0 条
🔍 还需 5000 条样本
📦 从 0Nm_BPFO_30.csv 抽取 5000 条。
🎉 类别 7 已抽满 5000 条。

🎯 开始处理类别 8，目标抽取 5000 条
📊 当前类别 8 已有 0 条
🔍 还需 5000 条样本
📦 从 0Nm_Misalign_01.csv 抽取 5000 条。
🎉 类别 8 已抽满 5000 条。

🎯 开始处理类别 9，目标抽取 5000 条
📊 当前类别 9 已有 0 条
🔍 还需 5000 条样本
📦 从 0Nm_Misalign_03.csv 抽取 5000 条。
🎉 类别 9 已抽满 5000 条。

🎯 开始处理类别 10，目标抽取 5000 条
📊 当前类别 10 已有 0

### 训练集、测试集、验证集划分，另存为新的csv文件

In [2]:
import os
import pandas as pd
import numpy as np

# ============= 1. 路径设置 =============
root_dir = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/all_class_morenode"
csv_path = os.path.join(root_dir, "mixed_allclass.csv")

print("加载数据：", csv_path)

train_dir = os.path.join(root_dir, "train")
val_dir = os.path.join(root_dir, "val")
test_dir = os.path.join(root_dir, "test")

os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# ============= 2. 读取数据 =============
df = pd.read_csv(csv_path)
print(f"数据加载成功，共 {len(df)} 条记录")

# 记录原始索引顺序
original_index = df.index.to_numpy()

# ============= 3. 设置划分比例 =============
train_ratio = 0.6
val_ratio = 0.2
test_ratio = 0.2
total = len(df)

# ============= 4. 随机抽取索引，不打乱顺序 =============
rng = np.random.default_rng(42)

# 随机采样，但不改变原始顺序
all_indices = np.arange(total)
rng.shuffle(all_indices)

train_end = int(total * train_ratio)
val_end = train_end + int(total * val_ratio)

train_idx = np.sort(all_indices[:train_end])
val_idx = np.sort(all_indices[train_end:val_end])
test_idx = np.sort(all_indices[val_end:])

# ============= 5. 保持原顺序抽取数据 =============
train_df = df.loc[train_idx].copy()
val_df = df.loc[val_idx].copy()
test_df = df.loc[test_idx].copy()

# ============= 6. 删除最后一列（source_file） =============
last_col = df.columns[-1]
print(f"移除最后一列：{last_col}")

train_df = train_df.iloc[:, :-1]
val_df = val_df.iloc[:, :-1]
test_df = test_df.iloc[:, :-1]

# ============= 7. 保存 CSV =============
train_df.to_csv(os.path.join(train_dir, "train.csv"), index=False)
val_df.to_csv(os.path.join(val_dir, "val.csv"), index=False)
test_df.to_csv(os.path.join(test_dir, "test.csv"), index=False)

print("所有数据集已保存（保持原顺序，已去除最后一列）。")

# ============= 8. 保存日志 =============
log_path = os.path.join(root_dir, "split_log.txt")
with open(log_path, "w") as f:
    f.write("KAIST all_class 数据集随机划分日志（保持原顺序+忽略最后一列）\n\n")
    f.write(f"总样本数: {len(df)}\n")
    f.write(f"Train: {len(train_df)}\n")
    f.write(f"Val:   {len(val_df)}\n")
    f.write(f"Test:  {len(test_df)}\n\n")
    f.write("划分比例:\n")
    f.write(f"train_ratio = {train_ratio}\n")
    f.write(f"val_ratio   = {val_ratio}\n")
    f.write(f"test_ratio  = {test_ratio}\n")

print("日志写入完成。")


加载数据： /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/all_class_morenode/mixed_allclass.csv
数据加载成功，共 70000 条记录
移除最后一列：source_file
所有数据集已保存（保持原顺序，已去除最后一列）。
日志写入完成。


### mixed_data->pyg

#### 时间邻点建图

In [1]:
# ================== 1️⃣ 导入路径与函数 ==================
%run ../_init_path.py
from build_pyg_data import build_local_temporal_graph   # ★ 你要用的函数
import pandas as pd
import os
import torch

# ================== 2️⃣ 根目录 ==================
root_dir = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/all_class_morenode"

splits = {
    "train": os.path.join(root_dir, "train/train.csv"),
    "val":   os.path.join(root_dir, "val/val.csv"),
    "test":  os.path.join(root_dir, "test/test.csv"),
}

# ================== 3️⃣ 建图参数可调 ==================
num_edges = 100     # 时间相邻 KNN 数（你自己定）

# ================== 4️⃣ 分别为三个集建图 ==================
for split_name, csv_path in splits.items():

    print(f"\n🚀 开始构建 {split_name} 图结构...")
    assert os.path.exists(csv_path), f"❌ CSV 文件不存在：{csv_path}"

    # === 保存路径：train/pyg/、val/pyg/、test/pyg/ ===
    pyg_dir = os.path.join(os.path.dirname(csv_path), "pyg")
    os.makedirs(pyg_dir, exist_ok=True)

    print(f"📁 pyg 保存目录: {pyg_dir}")

    # === 调用本地时序建图函数 ===
    nodes_csv, edges_csv, graph_pt = build_local_temporal_graph(
        csv_path=csv_path,
        save_dir=pyg_dir,
        num_edges=num_edges
    )

    print(f"   📄 nodes_csv: {nodes_csv}")
    print(f"   📄 edges_csv: {edges_csv}")
    print(f"   📦 graph_pt : {graph_pt}")

    print(f"✅ {split_name} 图构建完成。")

print("\n🎉 三个数据集 (train/val/test) 的图已经全部独立构建完毕！")


✅ 已添加路径：/home/charles/HZU/Industrial_Software_Testing/Industrial_Software_Testing/my_CIL_V1/data_process/KAIST
✅ 已添加 my_lib：/home/charles/HZU/Industrial_Software_Testing/Industrial_Software_Testing/my_CIL_V1/data_process/KAIST/my_lib

🚀 开始构建 train 图结构...
📁 pyg 保存目录: /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/all_class_morenode/train/pyg
✅ 图结构构建完成，共 42000 个节点，4200000 条无向边
📁 nodes.csv: /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/all_class_morenode/train/pyg/nodes.csv
📁 edges.csv: /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/all_class_morenode/train/pyg/edges.csv
📁 graph.pt : /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/all_class_morenode/train/pyg/graph.pt
🧩 特征维度: 4 (已自动忽略首列与标签列)
   📄 nodes_csv: /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/all_class_morenode/train/pyg/nodes.csv
   📄 edges_csv: /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/all_class_morenode/train/pyg/edges.csv
   📦 graph_pt : /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/all_class_morenode/tr

In [4]:
# ================== 1️⃣ 导入路径与函数 ==================
%run ../_init_path.py

import torch
import os
import numpy as np

# ================== 2️⃣ 读取 train 图 ==================
train_graph_path = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/all_class_morenode/train/pyg/graph.pt"
assert os.path.exists(train_graph_path), f"❌ 文件不存在：{train_graph_path}"

print(f"📌 加载 train 图：{train_graph_path}")
data = torch.load(train_graph_path)

total_nodes = data.x.size(0)
print(f"节点数 = {total_nodes}")

# ================== 3️⃣ 设置比例 ==================
train_label_ratio = 0.3   # 30% 节点为 train_withlabel

num_withlabel = int(total_nodes * train_label_ratio)
num_nolabel = total_nodes - num_withlabel

print(f"👉 有标签节点：{num_withlabel}  | 无标签节点：{num_nolabel}")

# ================== 4️⃣ 随机生成 mask（保持可复现） ==================
rng = np.random.default_rng(42)

all_indices = np.arange(total_nodes)
rng.shuffle(all_indices)

withlabel_idx = all_indices[:num_withlabel]
nolabel_idx   = all_indices[num_withlabel:]

# 创建 mask
train_withlabel_mask = np.zeros(total_nodes, dtype=bool)
train_nolabel_mask   = np.zeros(total_nodes, dtype=bool)

train_withlabel_mask[withlabel_idx] = True
train_nolabel_mask[nolabel_idx] = True

# ================== 5️⃣ 添加到 Data 对象 ==================
data.train_withlabel_mask = torch.tensor(train_withlabel_mask)
data.train_nolabel_mask   = torch.tensor(train_nolabel_mask)

print("✅ 掩码添加完成：")
print(f"  train_withlabel_mask.sum() = {int(data.train_withlabel_mask.sum())}")
print(f"  train_nolabel_mask.sum()   = {int(data.train_nolabel_mask.sum())}")

# ================== 6️⃣ 保存新的图结构 ==================
save_path = os.path.join(
    os.path.dirname(train_graph_path),
    "train_graph_with_labelmask.pt"
)

torch.save(data, save_path)
print(f"🎉 新图已保存为：{save_path}")


✅ 已添加路径：/home/charles/HZU/Industrial_Software_Testing/Industrial_Software_Testing/my_CIL_V1/data_process/KAIST
✅ 已添加 my_lib：/home/charles/HZU/Industrial_Software_Testing/Industrial_Software_Testing/my_CIL_V1/data_process/KAIST/my_lib
📌 加载 train 图：/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/all_class_morenode/train/pyg/graph.pt
节点数 = 42000
👉 有标签节点：12600  | 无标签节点：29400
✅ 掩码添加完成：
  train_withlabel_mask.sum() = 12600
  train_nolabel_mask.sum()   = 29400


/tmp/ipykernel_798073/3034512645.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(train_graph_path)


🎉 新图已保存为：/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/all_class_morenode/train/pyg/train_graph_with_labelmask.pt


#### 高相似度类别的建图,滑动窗口建图

In [14]:
# ================== 1️⃣ 导入库 ==================
import pandas as pd
import os
from sklearn.model_selection import train_test_split

# ================== 2️⃣ 原始 CSV 路径 ==================
input_csv = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/mixed_allclass_nofinal.csv"
assert os.path.exists(input_csv), f"❌ 输入文件不存在：{input_csv}"

# ================== 3️⃣ 输出根目录 ==================
output_root = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/all_class_morenode_high_similar"
os.makedirs(output_root, exist_ok=True)

# ================== 4️⃣ 读取数据 ==================
df = pd.read_csv(input_csv)
print(f"📌 原始数据量: {len(df)}")

# ================== 5️⃣ 筛选 label {1,11,12,13,14} ==================
target_labels = [1, 11, 12, 13, 14]
df_filtered = df[df["label"].isin(target_labels)]
print(f"📌 筛选后数据量: {len(df_filtered)}")

# 保存筛选后的数据
filtered_csv_path = os.path.join(output_root, "mixed_high_similar.csv")
df_filtered.to_csv(filtered_csv_path, index=False)
print(f"✅ 已保存筛选后的数据：{filtered_csv_path}")

# ================== 6️⃣ 随机划分 (6 : 2 : 2)，但保持索引 ==================
# step1: 先取得所有 index（不打乱原 df_filtered 秩序）
indices = df_filtered.index.to_numpy()

# step2: 随机划分 indices（真正的随机性在这里）
train_idx, temp_idx = train_test_split(
    indices, test_size=0.4, random_state=42
)
val_idx, test_idx = train_test_split(
    temp_idx, test_size=0.5, random_state=42
)

# step3: 用 iloc 根据 index 选数据，但不打乱，保持原 df 顺序
train_df = df_filtered.loc[sorted(train_idx)]
val_df   = df_filtered.loc[sorted(val_idx)]
test_df  = df_filtered.loc[sorted(test_idx)]

print(f"📌 Train: {len(train_df)}, Val: {len(val_df)}, Test: {len(test_df)}")

# ================== 7️⃣ 保存 ==================
train_dir = os.path.join(output_root, "train")
val_dir   = os.path.join(output_root, "val")
test_dir  = os.path.join(output_root, "test")

os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

train_df.to_csv(os.path.join(train_dir, "train.csv"), index=False)
val_df.to_csv(os.path.join(val_dir, "val.csv"), index=False)
test_df.to_csv(os.path.join(test_dir, "test.csv"), index=False)

print("\n✅ Train/Val/Test 已保存（随机划分，但顺序保持不变）：")
print(f"   🟩 Train: {os.path.join(train_dir, 'train.csv')}")
print(f"   🟦 Val  : {os.path.join(val_dir, 'val.csv')}")
print(f"   🟥 Test : {os.path.join(test_dir, 'test.csv')}")

print("\n🎉 随机划分 + 顺序保存 6:2:2 完成！")


📌 原始数据量: 22700
📌 筛选后数据量: 7500
✅ 已保存筛选后的数据：/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/all_class_morenode_high_similar/mixed_high_similar.csv
📌 Train: 4500, Val: 1500, Test: 1500

✅ Train/Val/Test 已保存（随机划分，但顺序保持不变）：
   🟩 Train: /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/all_class_morenode_high_similar/train/train.csv
   🟦 Val  : /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/all_class_morenode_high_similar/val/val.csv
   🟥 Test : /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/all_class_morenode_high_similar/test/test.csv

🎉 随机划分 + 顺序保存 6:2:2 完成！


In [15]:
# ================== 1️⃣ 导入路径与函数 ==================
%run ../_init_path.py
from build_pyg_data import build_local_temporal_graph   # ★ 你要用的函数
import pandas as pd
import os
import torch

# ================== 2️⃣ 根目录 ==================
root_dir = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/all_class_morenode_high_similar"

splits = {
    "train": os.path.join(root_dir, "train/train.csv"),
    "val":   os.path.join(root_dir, "val/val.csv"),
    "test":  os.path.join(root_dir, "test/test.csv"),
}

# ================== 3️⃣ 建图参数可调 ==================
num_edges = 1000     # 时间相邻 KNN 数（你自己定）

# ================== 4️⃣ 分别为三个集建图 ==================
for split_name, csv_path in splits.items():

    print(f"\n🚀 开始构建 {split_name} 图结构...")
    assert os.path.exists(csv_path), f"❌ CSV 文件不存在：{csv_path}"

    # === 保存路径：train/pyg/、val/pyg/、test/pyg/ ===
    pyg_dir = os.path.join(os.path.dirname(csv_path), "pyg")
    os.makedirs(pyg_dir, exist_ok=True)

    print(f"📁 pyg 保存目录: {pyg_dir}")

    # === 调用本地时序建图函数 ===
    nodes_csv, edges_csv, graph_pt = build_local_temporal_graph(
        csv_path=csv_path,
        save_dir=pyg_dir,
        num_edges=num_edges
    )

    print(f"   📄 nodes_csv: {nodes_csv}")
    print(f"   📄 edges_csv: {edges_csv}")
    print(f"   📦 graph_pt : {graph_pt}")

    print(f"✅ {split_name} 图构建完成。")

print("\n🎉 三个数据集 (train/val/test) 的图已经全部独立构建完毕！")


✅ 已添加路径：/home/charles/HZU/Industrial_Software_Testing/Industrial_Software_Testing/my_CIL_V1/data_process/KAIST
✅ 已添加 my_lib：/home/charles/HZU/Industrial_Software_Testing/Industrial_Software_Testing/my_CIL_V1/data_process/KAIST/my_lib

🚀 开始构建 train 图结构...
📁 pyg 保存目录: /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/all_class_morenode_high_similar/train/pyg
✅ 图结构构建完成，共 4500 个节点，4500000 条无向边
📁 nodes.csv: /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/all_class_morenode_high_similar/train/pyg/nodes.csv
📁 edges.csv: /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/all_class_morenode_high_similar/train/pyg/edges.csv
📁 graph.pt : /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/all_class_morenode_high_similar/train/pyg/graph.pt
🧩 特征维度: 4 (已自动忽略首列与标签列)
   📄 nodes_csv: /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/all_class_morenode_high_similar/train/pyg/nodes.csv
   📄 edges_csv: /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/all_class_morenode_high_similar/train/pyg/edges.csv
   📦 grap

In [16]:
# ================== 1️⃣ 导入路径与函数 ==================
%run ../_init_path.py

import torch
import os
import numpy as np

# ================== 2️⃣ 读取 train 图 ==================
train_graph_path = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/all_class_morenode_high_similar/train/pyg/graph.pt"
assert os.path.exists(train_graph_path), f"❌ 文件不存在：{train_graph_path}"

print(f"📌 加载 train 图：{train_graph_path}")
data = torch.load(train_graph_path)

total_nodes = data.x.size(0)
print(f"节点数 = {total_nodes}")

# ================== 3️⃣ 设置比例 ==================
train_label_ratio = 0.3   # 30% 节点为 train_withlabel

num_withlabel = int(total_nodes * train_label_ratio)
num_nolabel = total_nodes - num_withlabel

print(f"👉 有标签节点：{num_withlabel}  | 无标签节点：{num_nolabel}")

# ================== 4️⃣ 随机生成 mask（保持可复现） ==================
rng = np.random.default_rng(42)

all_indices = np.arange(total_nodes)
rng.shuffle(all_indices)

withlabel_idx = all_indices[:num_withlabel]
nolabel_idx   = all_indices[num_withlabel:]

# 创建 mask
train_withlabel_mask = np.zeros(total_nodes, dtype=bool)
train_nolabel_mask   = np.zeros(total_nodes, dtype=bool)

train_withlabel_mask[withlabel_idx] = True
train_nolabel_mask[nolabel_idx] = True

# ================== 5️⃣ 添加到 Data 对象 ==================
data.train_withlabel_mask = torch.tensor(train_withlabel_mask)
data.train_nolabel_mask   = torch.tensor(train_nolabel_mask)

print("✅ 掩码添加完成：")
print(f"  train_withlabel_mask.sum() = {int(data.train_withlabel_mask.sum())}")
print(f"  train_nolabel_mask.sum()   = {int(data.train_nolabel_mask.sum())}")

# ================== 6️⃣ 保存新的图结构 ==================
save_path = os.path.join(
    os.path.dirname(train_graph_path),
    "train_graph_with_labelmask.pt"
)

torch.save(data, save_path)
print(f"🎉 新图已保存为：{save_path}")


✅ 已添加路径：/home/charles/HZU/Industrial_Software_Testing/Industrial_Software_Testing/my_CIL_V1/data_process/KAIST
✅ 已添加 my_lib：/home/charles/HZU/Industrial_Software_Testing/Industrial_Software_Testing/my_CIL_V1/data_process/KAIST/my_lib
📌 加载 train 图：/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/all_class_morenode_high_similar/train/pyg/graph.pt
节点数 = 4500
👉 有标签节点：1350  | 无标签节点：3150
✅ 掩码添加完成：
  train_withlabel_mask.sum() = 1350
  train_nolabel_mask.sum()   = 3150


/tmp/ipykernel_32966/3477331405.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(train_graph_path)


🎉 新图已保存为：/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/all_class_morenode_high_similar/train/pyg/train_graph_with_labelmask.pt


#### 余弦相似度建图

In [ ]:
# # ================== 1️⃣ 导入路径与函数 ==================
# %run ../_init_path.py
# from build_pyg_data import build_similarity_knn_graph
# import pandas as pd
# import torch
# import os

# # ================== 2️⃣ 参数设置 ==================
# csv_path = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/mixed_allclass_nofinal.csv"
# save_dir = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/pyg_data/cos_knn"

# # ================== 3️⃣ 调用建图函数 ==================
# print("🚀 开始构建基于余弦相似度的KNN图 ...")
# nodes_csv, edges_csv, graph_pt = build_similarity_knn_graph(
#     csv_path=csv_path,
#     save_dir=save_dir,
#     num_edges=10
# )
# print("\n✅ 图结构已生成：")
# print(f"📁 节点文件: {nodes_csv}")
# print(f"📁 边文件:   {edges_csv}")
# print(f"📁 图文件:   {graph_pt}")

# # ================== 4️⃣ 验证生成的图数据 ==================
# print("\n🧩 验证生成的 PyG 图数据结构 ...")

# # 读取 PyG 对象
# data = torch.load(graph_pt, weights_only=False)

# # === 基础信息 ===
# num_nodes = data.x.size(0)
# num_edges = data.edge_index.size(1)
# num_features = data.x.size(1)
# has_label = hasattr(data, "y") and data.y is not None

# print(f"📦 Data 对象结构: {data}")
# print(f"🧮 节点数: {num_nodes}")
# print(f"🧮 边数: {num_edges // 2}（无向边）")
# print(f"🧩 特征维度: {num_features}")
# print(f"🏷️  标签存在: {'✅ 是' if has_label else '❌ 否'}")

# # === 标签验证 ===
# if has_label:
#     y = data.y.cpu().numpy()
#     unique_labels = sorted(set(y))
#     print(f"🎯 标签类别数量: {len(unique_labels)} | 类别: {unique_labels[:10]}")
#     print(f"前5个标签: {y[:5]}")
# else:
#     print("⚠️ 未检测到 data.y，标签列可能缺失或未正确指定。")

# # === 节点/边文件验证 ===
# print("\n📊 验证节点和边 CSV 文件 ...")
# df_nodes = pd.read_csv(nodes_csv)
# df_edges = pd.read_csv(edges_csv)

# print(f"✅ 节点文件行数: {len(df_nodes)} | 与图节点数 {'一致 ✅' if len(df_nodes)==num_nodes else '不一致 ⚠️'}")
# print(f"✅ 边文件行数: {len(df_edges)} | 每两行对应一条无向边")

# if "source" in df_edges.columns and "target" in df_edges.columns:
#     unique_nodes = pd.concat([df_edges["source"], df_edges["target"]]).nunique()
#     print(f"🧮 边中涉及的唯一节点数: {unique_nodes}")
# else:
#     print("⚠️ 边文件缺少 'source' 或 'target' 列！")

# print("\n🎯 图数据验证完成。")


In [ ]:
# # ================== 1️⃣ 导入路径与函数 ==================
# %run ../_init_path.py
# from build_pyg_data import add_random_masks_with_label_split
# import pandas as pd
# import torch
# import os

# # ================== 2️⃣ 参数设置 ==================
# ratios = {"train": 0.6, "val": 0.2, "test": 0.2}

# graph_path = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/pyg_data/cos_knn/mixed_allclass_nofinal_graph.pt"
# save_path  = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/pyg_data/cos_knn/mixed_allclass_arabic_graph_withmask.pt"

# # ================== 3️⃣ 生成掩码 ==================
# print("🚀 开始为图添加随机掩码 ...")
# new_graph_path = add_random_masks_with_label_split(
#     graph_path=graph_path,
#     save_path=save_path,     # 若想覆盖源文件则直接传 None
#     ratios=ratios,
#     train_label_ratio=0.4,   # 训练集中40%为有标签
# )
# print(f"\n✅ 掩码生成与保存完成：{new_graph_path}")

# # ================== 4️⃣ 验证生成结果 ==================
# print("\n🧩 验证生成的图结构与掩码 ...")

# # 读取 PyG 数据
# data = torch.load(new_graph_path, weights_only=False)

# # === 基本信息 ===
# print(f"\n📦 图数据结构: {data}")
# print(f"x shape: {data.x.shape}, y shape: {data.y.shape}")
# print(f"前5个标签: {data.y[:5].tolist()}")

# # === 掩码存在性检测 ===
# mask_attrs = ["train_mask", "val_mask", "test_mask", "train_withlabel_mask", "train_nolabel_mask"]
# print("\n📋 掩码存在性检查：")
# for attr in mask_attrs:
#     print(f"  {attr:<25}: {'✅ 存在' if hasattr(data, attr) else '❌ 不存在'}")

# # === 掩码统计 ===
# if hasattr(data, "train_mask"):
#     total_nodes = data.x.size(0)
#     n_train = int(data.train_mask.sum())
#     n_val   = int(data.val_mask.sum()) if hasattr(data, "val_mask") else 0
#     n_test  = int(data.test_mask.sum()) if hasattr(data, "test_mask") else 0
#     n_with  = int(data.train_withlabel_mask.sum()) if hasattr(data, "train_withlabel_mask") else 0
#     n_nol   = int(data.train_nolabel_mask.sum()) if hasattr(data, "train_nolabel_mask") else 0

#     print("\n📊 掩码数量验证：")
#     print(f"  train_mask         = {n_train}")
#     print(f"  val_mask           = {n_val}")
#     print(f"  test_mask          = {n_test}")
#     print(f"  └─ train_withlabel = {n_with}")
#     print(f"  └─ train_nolabel   = {n_nol}")
#     print(f"  合计节点数         = {total_nodes}")
#     if n_train + n_val + n_test == total_nodes:
#         print("✅ 掩码划分总数匹配节点总数")
#     else:
#         print("⚠️ 注意：掩码划分节点数与总数不完全匹配，可能存在舍入误差。")

# # === 标签统计 ===
# unique_labels = sorted(set(data.y.cpu().numpy().tolist()))
# print("\n🎯 标签分布检查：")
# print(f"  标签类别数: {len(unique_labels)}")
# print(f"  类别示例: {unique_labels[:10]}")

# print("\n🎯 掩码与标签验证完成。")


#### time + cos_knn 融合建图